# 職缺資料清理與寫入 Supabase

依 `cleaner步驟_v2.md` 清理 `jobs_rows.csv`，並依 ERD（`career_pilot_ERD_欄位對齊總表.md`）寫入 `company_info`、`job_posting`。

**執行順序**：請由上面依序跑到下面，可逐步執行各階段並測試。  
**工作目錄**：Notebook 請在 `supabase_control` 下開啟，確保 `jobs_rows.csv`、`Erd/.env` 路徑正確。

## 前置準備：安裝套件、載入資料

- 套件：`pandas`, `numpy`, `supabase`, `python-dotenv`（見 `pyproject.toml`）
- 資料：`jobs_rows.csv`

In [38]:
import pandas as pd
import numpy as np
import re
import json
from pathlib import Path

DATA_DIR = Path(".")
RAW_CSV = DATA_DIR / "jobs_rows.csv"

df_raw = pd.read_csv(RAW_CSV)
print(f"✓ 載入 {len(df_raw):,} 筆原始資料")
print(f"欄位: {list(df_raw.columns)}")
df_raw.head(2)

✓ 載入 7,839 筆原始資料
欄位: ['job_id', 'job_name', 'company_name', 'company_url', 'update_date', 'url', 'status', 'job_description', 'job_category', 'salary', 'job_type', 'location', 'management', 'business_trip', 'work_time', 'vacation', 'start_work', 'headcount', 'work_exp', 'education', 'major', 'language', 'skills', 'tools', 'certificates', 'other_requirements', 'legal_benefits', 'other_benefits', 'raw_benefits', 'contact_info', 'actively_hiring', 'applicants', 'created_at']


,job_id,job_name,company_name,company_url,update_date,url,status,job_description,job_category,salary,...,tools,certificates,other_requirements,legal_benefits,other_benefits,raw_benefits,contact_info,actively_hiring,applicants,created_at
0,109g12026-01-21,C#軟體工程師,泛太資訊科技開發股份有限公司,NaN,2026-01-21,https://www.104.com.tw/job/109g1?jobsource=job...,active,- 熟悉C#、.NET 、ASP.Net\n-依據系統需求規格進行程式設計與開發\n-使用 ...,軟體工程師、Internet程式設計師、系統分析師,待遇面議,...,Windows 2003、Windows XP、ASP.NET、C#、Visual Basi...,NaN,NaN,週休二日、勞保、健保、職災保險,年終獎金、三節獎金/禮品、專業證照獎金、結婚禮金、生育津貼、員工進修補助、國內旅遊、部門聚餐...,泛太資訊遵循勞基法之相關規定並提供下列福利\n\n►黃金地段辦公室！國父紀念館站１號出口，輕...,陳小姐\n02-27313250-202\n本職務設定3個工作天回覆,False,0~5,2026-01-28 09:02:28.359263+00
1,1x89f2026-01-26,研發替代役-軟體工程師,凌群電腦股份有限公司,NaN,2026-01-26,https://www.104.com.tw/job/1x89f?jobsource=job...,active,軟體開發與設計、架構設計、資料設計、系統整合、軟體產品研發、技術管理、專案管理\n\n※薪資...,軟體工程師、系統分析師、Internet程式設計師,"月薪37,000元以上取得專屬你的薪水報告",...,NaN,NaN,1.研究所以上資訊相關系所畢\n2.熟稔Java或.Net語言\n3.具系統分析設計與程式撰...,哺乳室、週休二日、家庭照顧假、勞保、健保、陪產假、產假、特別休假、育嬰留停、女性生理假、勞退...,生育津貼、社團補助、員工進修補助、部門聚餐、社團活動、特約商店、內部講師鐘點費、員工團體保險,人才是凌群最重要的資產 \n用心照護。安心工作\n\n【凌群福利措施】\n\n★完善的保險保...,李小姐\n誠摯邀請您至公司網站www.syscom.com.tw 更進一步認識凌群，並期待與...,False,0~5,2026-01-28 09:26:33.981361+00


## 階段一：資料探索與品質檢查

- 缺失值統計、重複檢查、job_id 格式、公司名稱統計、薪資檢查
- **headcount**：職缺需求人數，非公司規模，清理時不解析

In [39]:
# 1. 缺失值統計
missing = df_raw.isna().sum()
missing_pct = (missing / len(df_raw) * 100).round(1)
miss_df = pd.DataFrame({"缺失數": missing, "缺失%": missing_pct})
print("【缺失值統計】")
display(miss_df[miss_df["缺失數"] > 0].sort_values("缺失數", ascending=False))

# 2. 重複檢查
dup_all = df_raw.duplicated().sum()
dup_cnj = df_raw.duplicated(subset=["company_name", "job_name"], keep="first").sum()
has_url = df_raw["url"].notna() & (df_raw["url"].astype(str).str.len() > 0)
dup_url = df_raw.loc[has_url].duplicated(subset=["url", "job_name"], keep="first").sum()
print(f"\n【重複】 完全重複: {dup_all:,} | (company_name, job_name): {dup_cnj:,} | (url, job_name) 有 url 時: {dup_url:,}")

# 3. job_id 格式（異常範例）
jid = df_raw["job_id"].astype(str)
weird = jid[~jid.str.match(r"^[a-zA-Z0-9\-]+$", na=False)]
print(f"\n【job_id 異常】 {len(weird):,} 筆")
if len(weird) > 0:
    display(weird.head())

# 4. 公司統計
n_company = df_raw["company_name"].nunique()
top = df_raw["company_name"].value_counts().head(5)
print(f"\n【公司】 不重複 {n_company:,} 家；職缺數前 5:")
display(top)

# 5. 薪資範例
print("\n【薪資欄 salary 範例】")
display(df_raw["salary"].dropna().head(10))

【缺失值統計】


,缺失數,缺失%
company_url,7572,96.6
certificates,7548,96.3
skills,3657,46.7
other_requirements,3000,38.3
other_benefits,2542,32.4
legal_benefits,2340,29.9
tools,2043,26.1
raw_benefits,521,6.6
business_trip,405,5.2
management,405,5.2



【重複】 完全重複: 0 | (company_name, job_name): 369 | (url, job_name) 有 url 時: 321

【job_id 異常】 0 筆

【公司】 不重複 3,275 家；職缺數前 5:


company_name
台達電子工業股份有限公司 _DELTA ELECTRONICS INC.    57
鴻海精密工業股份有限公司                            50
工研院 _財團法人工業技術研究院                        50
國泰世華商業銀行股份有限公司                          50
華碩電腦股份有限公司                              38
Name: count, dtype: int64


【薪資欄 salary 範例】


0                          待遇面議
1         月薪37,000元以上取得專屬你的薪水報告
2    月薪35,000~50,000元取得專屬你的薪水報告
3    月薪35,000~75,000元取得專屬你的薪水報告
4                          待遇面議
5                          待遇面議
6                          待遇面議
7         月薪36,000元以上取得專屬你的薪水報告
8                          待遇面議
9                          待遇面議
Name: salary, dtype: str

## 階段二：資料清理函數定義

依步驟定義：`clean_text`、`extract_industry`(公司名稱)、`infer_industry_from_job_category`(職缺類別 fallback)、`standardize_location`→(full_address, city, district)、`clean_salary`、`determine_remote_option`、`merge_requirements`、`create_job_details`。

In [40]:
def clean_text(text):
    """清理文字：移除多餘空白、換行、特殊字元；處理 None/NaN。"""
    if pd.isna(text) or text is None:
        return None
    s = str(text).strip()
    s = re.sub(r"[\r\n]+", " ", s)
    s = re.sub(r"\s+", " ", s)
    return s if s else None


# 組織後綴：移除後再進行關鍵字匹配
ORG_SUFFIXES = ["股份有限公司", "有限公司", "集團", "分公司", "財團法人"]

def clean_company_name(company_name):
    """移除公司名稱中的組織後綴（股份有限公司、有限公司、集團、分公司等）。"""
    if pd.isna(company_name) or not str(company_name).strip():
        return None
    name = str(company_name).strip()
    for suffix in ORG_SUFFIXES:
        if name.endswith(suffix):
            name = name[: -len(suffix)].strip()
        while suffix in name:
            name = name.replace(suffix, " ").strip()
    return name if name else None


# 分層關鍵字優先級：特定領域 > 複合領域 > 通用關鍵字
PRIORITY_LAYERS = {
    "tier_1_specific": [
        ("製造業", ["半導體製造", "製程", "產線", "封裝測試", "晶圓", "fab", "光電", "台積電"]),
        ("醫療", ["醫療器材", "醫藥", "生技", "藥廠", "醫檢", "診所", "醫院", "醫材", "醫電", "生醫", "製藥", "藥業"]),
        ("金融業", ["銀行", "保險", "證券", "投信", "金控", "金融", "控股", "資產管理"]),
    ],
    "tier_2_mixed": [
        ("資訊科技", ["軟體", "網路服務", "雲端", "ai應用", "資安", "系統整合"]),
        ("製造業", ["材料", "精密", "機械"]),
    ],
    "tier_3_generic": [
        ("資訊科技", ["科技", "資訊", "數位"]),
        ("服務業", ["服務"]),
    ],
}


def extract_industry(company_name, job_category=None):
    """從公司名稱推斷產業類別（分層關鍵字優先級，向後相容保留原名）。"""
    if pd.isna(company_name) or not str(company_name).strip():
        return None
    name = clean_company_name(company_name)
    if not name:
        return None
    name_lower = name.lower()
    # Tier 1：匹配即直接返回（人力銀行為求職平台，不以「銀行」判為金融業）
    for industry, keywords in PRIORITY_LAYERS["tier_1_specific"]:
        kws = [kw for kw in keywords if not (industry == "金融業" and kw == "銀行" and "人力銀行" in name_lower)]
        if any(kw in name_lower for kw in kws):
            return industry
    # Tier 2、3：記錄候選，優先取 Tier2
    for industry, keywords in PRIORITY_LAYERS["tier_2_mixed"]:
        if any(kw in name_lower for kw in keywords):
            return industry
    for industry, keywords in PRIORITY_LAYERS["tier_3_generic"]:
        if any(kw in name_lower for kw in keywords):
            return industry
    if job_category is not None and str(job_category).strip():
        return infer_industry_from_job_category(job_category)
    return None


def infer_industry_from_job_category(job_categories_text):
    """從職缺類別（job_category 彙總字串）推斷產業；公司名稱無匹配時 fallback 用。"""
    if pd.isna(job_categories_text) or not str(job_categories_text).strip():
        return None
    s = str(job_categories_text).strip().lower()
    manufacturing_kw = [
        "製造", "產線", "設備", "製程", "機構工程", "半導體", "光電", "pcb", "smt",
        "品保", "倉管", "生產", "焊接", "cnc", "製程工程師", "設備工程師", "生產管理",
        "品管", "qc", "qe", "ie", "me", "廠務", "生產線", "作業員",
    ]
    medical_kw = [
        "醫護", "護理", "藥師", "醫檢", "醫事", "醫師", "護理師", "醫檢師",
        "醫療器材", "醫學工程", "臨床", "復健",
    ]
    keywords = [
        ("資訊科技", ["軟體", "程式", "系統分析", "internet", "mis", "韌體", "資料庫", "資安", "演算法", "dba", "bios", "全端", "後端", "前端", "資料科學", "大數據", "雲端", "devops", "sre", "嵌入式", "網管", "qa"]),
        ("製造業", manufacturing_kw),
        ("金融業", ["金融", "銀行", "保險", "證券", "理財", "風控", "精算", "授信", "櫃員"]),
        ("醫療", medical_kw),
        ("行銷", ["行銷", "廣告", "媒體", "電商", "社群", "文案", "企劃", "數位行銷"]),
        ("商業", ["人資", "人力資源", "會計", "財務", "審計", "業務", "客服", "採購", "行政", "總務", "秘書", "法務", "顧問"]),
        ("設計", ["設計", "ui", "ux", "平面", "工業設計", "視覺"]),
        ("教育", ["教師", "講師", "教練", "補習", "教材", "教學"]),
    ]
    for industry, kws in keywords:
        if any(kw in s for kw in kws):
            return industry
    return None


import re

CITIES = [
    "台北市", "新北市", "桃園市", "台中市", "台南市", "高雄市", 
    "基隆市", "新竹市", "嘉義市",
    "新竹縣", "苗栗縣", "彰化縣", "南投縣", "雲林縣", "嘉義縣", 
    "屏東縣", "宜蘭縣", "花蓮縣", "台東縣", "澎湖縣", "金門縣", "連江縣",
]

# 定義區/鄉/鎮/市的正則表達式
DISTRICT_SUFFIX = re.compile(r"^(.+?[區鄉鎮市])")

VALID_DISTRICTS = {"東區", "北區", "香山區", "西區"}
GARDEN_BLACKLIST = ["工業園區", "科學園區", "園區", "太空中心"]  # 加入太空中心

def standardize_location(city, district, location):
    """輸出 (full_address, city, district)。"""
    parts = [x for x in [city, district, location] if pd.notna(x) and str(x).strip()]
    full = "".join(str(p).strip() for p in parts) if parts else None
    if not full:
        return None, None, None
    
    city_val, district_val = None, None
    
    for c in CITIES:
        if full.startswith(c):
            city_val = c
            rest = full[len(c):].strip()
            
            # 處理重複縣市名（如「新竹市新竹市...」）
            if rest.startswith(c):
                rest = rest[len(c):].strip()
            
            # 新竹市/嘉義市特殊處理
            if city_val in ["新竹市", "嘉義市"]:
                for valid_d in VALID_DISTRICTS:
                    if rest.startswith(valid_d):
                        district_val = valid_d
                        break
            
            # 一般縣市處理
            else:
                m = DISTRICT_SUFFIX.match(rest)
                if m:
                    candidate = m.group(1).strip()
                    if not any(b in candidate for b in GARDEN_BLACKLIST):
                        district_val = candidate
                
                if not district_val and rest:
                    tok = re.match(r"^([^\d路街段巷弄號]+?[區鄉鎮市])", rest)
                    if tok:
                        candidate = tok.group(1).strip()
                        if not any(b in candidate for b in GARDEN_BLACKLIST):
                            district_val = candidate
            break
    
    return full, city_val, district_val

def _parse_int(s):
    if not s:
        return None
    return int(re.sub(r"[,，\s]", "", str(s)))

def clean_salary(salary_raw):
    """薪資清理 → (min, max)。待遇面議→(40000,40000)；區間解析 min,max。"""
    if pd.isna(salary_raw) or not str(salary_raw).strip():
        return None, None
    s = str(salary_raw).strip()
    if "面議" in s or "面谈" in s:
        return 40000, 40000
    s_clean = re.sub(r"[,，\s]", "", s)
    mm = re.search(r"([\d,]+)\s*[~～\-－至到]\s*([\d,]+)", s)
    if mm:
        lo, hi = _parse_int(mm.group(1)), _parse_int(mm.group(2))
        if lo is not None and hi is not None:
            return (min(lo, hi), max(lo, hi))
    single = re.search(r"月薪\s*([\d,]+)", s)
    if single:
        v = _parse_int(single.group(1))
        if v is not None:
            return (v, v)
    num = re.search(r"(\d+)\s*元", s_clean)
    if num:
        v = int(num.group(1))
        return (v, 999999)
    return None, None


def determine_remote_option(addr, job_type):
    """判斷 remote / hybrid / onsite。可傳 full_address 與 job_type。"""
    if pd.notna(job_type) and "遠端" in str(job_type):
        return "remote"
    if pd.notna(job_type) and " hybrid" in str(job_type).lower():
        return "hybrid"
    addr_str = str(addr) if pd.notna(addr) else ""
    if "遠端" in addr_str or "remote" in addr_str.lower():
        return "remote"
    if "混合" in addr_str or "hybrid" in addr_str.lower():
        return "hybrid"
    return "onsite"


def merge_requirements(row):
    """合併 work_exp, education, major, language, skills, tools, certificates, other_requirements。"""
    keys = ["work_exp", "education", "major", "language", "skills", "tools", "certificates", "other_requirements"]
    parts = []
    for k in keys:
        v = row.get(k)
        if pd.notna(v) and str(v).strip():
            parts.append(str(v).strip())
    return "\n".join(parts) if parts else None


def create_job_details(row):
    """建立 job_details JSON。"""
    keys = ["work_time", "vacation", "start_work", "business_trip", "legal_benefits", "other_benefits", "raw_benefits"]
    d = {}
    for k in keys:
        v = row.get(k)
        if pd.notna(v) and str(v).strip():
            d[k] = str(v).strip()
    return d if d else None

print("✓ 清理函數已定義")

✓ 清理函數已定義


## 階段三：清理公司資料 (COMPANY_INFO)

依 `company_name` 分組、彙總 `job_category`；產業依序 **公司名稱** → **job_category fallback** → **未分類**。`company_size` / `location` 皆 NULL，對齊 ERD 後去重。

In [41]:
# 依 company_name 分組，彙總 job_category（供產業 fallback）
def _agg_job_cats(x):
    parts = x.dropna().astype(str).str.strip()
    parts = parts[parts.str.len() > 0].unique()
    return " | ".join(parts) if len(parts) else ""

company_agg = df_raw.groupby("company_name", as_index=False).agg(
    company_name=("company_name", "first"),
    job_categories=("job_category", _agg_job_cats),
)
company_agg["company_name"] = company_agg["company_name"].apply(clean_text)
company_agg = company_agg[company_agg["company_name"].notna() & (company_agg["company_name"].str.len() > 0)]

# 產業：公司名稱 → job_category 彙總 fallback → 未分類
def _resolve_industry(row):
    ind = extract_industry(row["company_name"])
    if ind is not None:
        return ind
    ind = infer_industry_from_job_category(row["job_categories"])
    if ind is not None:
        return ind
    return "未分類"

company_agg["industry"] = company_agg.apply(_resolve_industry, axis=1)
company_agg["company_size"] = None
company_agg["location"] = None
company_agg["website"] = None
company_agg["description"] = None

# 保留 job_categories 欄位並重命名為 job_category
df_company = company_agg[[
    "company_name", 
    "industry", 
    "job_categories",  # 新增：保留職缺類別欄位
    "company_size", 
    "location", 
    "website", 
    "description"
]].copy()

# 重命名欄位以符合資料庫欄位名稱
df_company.rename(columns={"job_categories": "job_category"}, inplace=True)

# 去重並重置索引
df_company = df_company.drop_duplicates(subset=["company_name"]).reset_index(drop=True)

print(f"✓ 公司主檔 {len(df_company):,} 家 | 有 industry（非未分類）: {(df_company['industry'] != '未分類').sum():,}")

# 驗證 job_category 欄位
print(f"✓ df_company 欄位: {list(df_company.columns)}")
print(f"✓ job_category 非空筆數: {df_company['job_category'].notna().sum()} / {len(df_company)}")
print("\n【job_category 範例】")
display(df_company[df_company['job_category'].notna()][['company_name', 'job_category']].head(3))

# industry 各產業類別比例（供檢查）
industry_dist = df_company["industry"].value_counts().sort_index()
industry_check = pd.DataFrame({
    "industry": industry_dist.index,
    "家數": industry_dist.values,
    "比例%": (industry_dist.values / len(df_company) * 100).round(2),
})
print("\n【industry 產業類別比例】")
display(industry_check)

# 顯示 job_category 範例（完整表格）
print("\n【job_category 範例（company_name, industry, job_category）】")
display(df_company[df_company['job_category'].notna()][['company_name', 'industry', 'job_category']].head(5))

✓ 公司主檔 3,275 家 | 有 industry（非未分類）: 3,137
✓ df_company 欄位: ['company_name', 'industry', 'job_category', 'company_size', 'location', 'website', 'description']
✓ job_category 非空筆數: 3275 / 3275

【job_category 範例】


,company_name,job_category
0,(Synaptics Taiwan)香港商新思國際科技有限公司台灣分公司,軟體工程師、全端工程師、通訊軟體工程師
1,(台泥)臺灣水泥股份有限公司,數據分析師／資料分析師、AI工程師、專案經理
2,(捷普集團)綠點高新科技股份有限公司捷普設計服務分公司,軟體工程師



【industry 產業類別比例】


,industry,家數,比例%
0,商業,25,0.76
1,教育,1,0.03
2,服務業,20,0.61
3,未分類,138,4.21
4,行銷,43,1.31
5,製造業,112,3.42
6,設計,14,0.43
7,資訊科技,2715,82.90
8,醫療,89,2.72
9,金融業,118,3.60



【job_category 範例（company_name, industry, job_category）】


,company_name,industry,job_category
0,(Synaptics Taiwan)香港商新思國際科技有限公司台灣分公司,資訊科技,軟體工程師、全端工程師、通訊軟體工程師
1,(台泥)臺灣水泥股份有限公司,未分類,數據分析師／資料分析師、AI工程師、專案經理
2,(捷普集團)綠點高新科技股份有限公司捷普設計服務分公司,資訊科技,軟體工程師
3,(總公司)南山人壽保險股份有限公司,金融業,軟體工程師、Internet程式設計師、系統分析師 | 軟體工程師、後端工程師 | 前端工程...
4,(美商)台灣通用器材股份有限公司,製造業,廠務


## 階段四：清理職缺資料 (JOB_POSTING)

清理標題、描述、合併要求、標準化地點 (full_address, city, district)、薪資、遠端選項、job_details、日期。去重（Upsert 模式）：有 `source_url` 依 **update_date 排序**，以 source_url 去重保留**最新一筆**；無 source_url 用 (company_name, job_title, full_address)。移除關鍵欄位空值。

In [42]:
# 複製並保留 company_name、job_category（稍後對應 company_id、寫入 job_posting）
df_jobs = df_raw.copy()
df_jobs["company_name"] = df_jobs["company_name"].apply(clean_text)
df_jobs["job_title"] = df_jobs["job_name"].apply(clean_text)
df_jobs["job_description"] = df_jobs["job_description"].apply(clean_text)
df_jobs["job_category"] = df_jobs["job_category"].apply(clean_text)  # 保留並清理職缺類別

# 合併要求
df_jobs["requirements"] = df_jobs.apply(merge_requirements, axis=1)

# 標準化地點：(full_address, city, district)。CSV 僅有 location
loc_out = df_jobs.apply(lambda r: standardize_location(None, None, r.get("location")), axis=1)
df_jobs["full_address"] = [x[0] for x in loc_out]
df_jobs["city"] = [x[1] for x in loc_out]
df_jobs["district"] = [x[2] for x in loc_out]

# 薪資
sal_out = df_jobs["salary"].apply(clean_salary)
df_jobs["salary_min"] = [x[0] for x in sal_out]
df_jobs["salary_max"] = [x[1] for x in sal_out]

# 遠端選項、job_details
df_jobs["remote_option"] = df_jobs.apply(lambda r: determine_remote_option(r.get("full_address"), r.get("job_type")), axis=1)
df_jobs["job_details"] = df_jobs.apply(create_job_details, axis=1)

# 日期：update_date -> posted_date, created_at -> scraped_at
df_jobs["posted_date"] = pd.to_datetime(df_jobs["update_date"], errors="coerce").dt.date
df_jobs["scraped_at"] = pd.to_datetime(df_jobs["created_at"], errors="coerce")

# 來源
df_jobs["source_platform"] = "104人力銀行"
df_jobs["source_url"] = df_jobs["url"].where(df_jobs["url"].notna() & (df_jobs["url"].astype(str).str.len() > 0))

# 固定欄位
df_jobs["is_active"] = True
df_jobs["is_embedded"] = False

print("✓ 職缺清理完成（未去重、未刪空）")
# 驗證 job_category 欄位
print(f"✓ df_jobs 欄位包含 job_category: {'job_category' in df_jobs.columns}")
print(f"✓ job_category 非空筆數: {df_jobs['job_category'].notna().sum()} / {len(df_jobs)}")
print("\n【job_category 範例】")
display(df_jobs[df_jobs["job_category"].notna()][["job_title", "job_category"]].head(5))
df_jobs[["company_name", "job_title", "job_category", "full_address", "city", "district", "salary_min", "salary_max", "source_url", "job_description"]].head()

✓ 職缺清理完成（未去重、未刪空）
✓ df_jobs 欄位包含 job_category: True
✓ job_category 非空筆數: 7572 / 7839

【job_category 範例】


,job_title,job_category
0,C#軟體工程師,軟體工程師、Internet程式設計師、系統分析師
1,研發替代役-軟體工程師,軟體工程師、系統分析師、Internet程式設計師
2,系統工程師(高雄),網路管理工程師、通訊軟體工程師、軟體工程師
3,資料庫程式工程師,軟體工程師、資料庫管理人員、MIS程式設計師
4,115年度研發替代役 - 網站後端工程師 Backend Engineer,軟體工程師、後端工程師


,company_name,job_title,job_category,full_address,city,district,salary_min,salary_max,source_url,job_description
0,泛太資訊科技開發股份有限公司,C#軟體工程師,軟體工程師、Internet程式設計師、系統分析師,台北市大安區光復南路102號7樓,台北市,大安區,40000.0,40000.0,https://www.104.com.tw/job/109g1?jobsource=job...,- 熟悉C#、.NET 、ASP.Net -依據系統需求規格進行程式設計與開發 -使用 C#...
1,凌群電腦股份有限公司,研發替代役-軟體工程師,軟體工程師、系統分析師、Internet程式設計師,台北市萬華區峨眉街115號6樓,台北市,萬華區,37000.0,37000.0,https://www.104.com.tw/job/1x89f?jobsource=job...,軟體開發與設計、架構設計、資料設計、系統整合、軟體產品研發、技術管理、專案管理 ※薪資依學經歷敘薪
2,瑞訊股份有限公司,系統工程師(高雄),網路管理工程師、通訊軟體工程師、軟體工程師,高雄市大社區萬金路385巷3-2號,高雄市,大社區,35000.0,50000.0,https://www.104.com.tw/job/1xpg2?jobsource=job...,PLC 或 SCADA (Supervisory Control And Data Acqu...
3,商智資訊股份有限公司,資料庫程式工程師,軟體工程師、資料庫管理人員、MIS程式設計師,台北市大安區復興南路2段363號3樓,台北市,大安區,35000.0,75000.0,https://www.104.com.tw/job/1yf7o?jobsource=job...,資料庫程式工程師 熟悉 SQL Server/Oracle/DB2 任一種資料庫操作
4,甲尚股份有限公司,115年度研發替代役 - 網站後端工程師 Backend Engineer,軟體工程師、後端工程師,新北市新店區寶橋路235巷126號2樓,新北市,新店區,40000.0,40000.0,https://www.104.com.tw/job/216lz?jobsource=job...,我們正在尋找熱愛挑戰的 .NET 後端工程師，你將負責高效能 API 開發、資料庫優化、行銷...


In [43]:
# 去重：有 source_url 以 update_date 排序，保留最新一筆（Upsert 模式）
has_url = df_jobs["source_url"].notna() & (df_jobs["source_url"].astype(str).str.len() > 0)
# 依 update_date（posted_date）降序，較新的在前，去重時 keep="first" 即保留最新
df_with_url_raw = df_jobs[has_url].copy()
df_with_url_raw = df_with_url_raw.sort_values("posted_date", ascending=False, na_position="last")
df_with_url = df_with_url_raw.drop_duplicates(subset=["source_url"], keep="first")
df_no_url = df_jobs[~has_url].drop_duplicates(subset=["company_name", "job_title", "full_address"], keep="last")
df_jobs = pd.concat([df_with_url, df_no_url], ignore_index=True)

# 移除關鍵欄位為空
df_jobs = df_jobs[
    df_jobs["company_name"].notna() & (df_jobs["company_name"].astype(str).str.len() > 0) &
    df_jobs["job_title"].notna() & (df_jobs["job_title"].astype(str).str.len() > 0) &
    df_jobs["job_description"].notna() & (df_jobs["job_description"].astype(str).str.len() > 0)
].copy()

n_url_empty = (df_jobs["source_url"].isna() | (df_jobs["source_url"].astype(str).str.strip().str.len() == 0)).sum()
print(f"✓ 去重並移除空值後 {len(df_jobs):,} 筆職缺 | source_url 為空: {n_url_empty:,} 筆")
print(f"✓ df_jobs 欄位: {list(df_jobs.columns)}")
print(f"✓ job_category 非空筆數: {df_jobs['job_category'].notna().sum()} / {len(df_jobs)}")

✓ 去重並移除空值後 7,252 筆職缺 | source_url 為空: 0 筆
✓ df_jobs 欄位: ['job_id', 'job_name', 'company_name', 'company_url', 'update_date', 'url', 'status', 'job_description', 'job_category', 'salary', 'job_type', 'location', 'management', 'business_trip', 'work_time', 'vacation', 'start_work', 'headcount', 'work_exp', 'education', 'major', 'language', 'skills', 'tools', 'certificates', 'other_requirements', 'legal_benefits', 'other_benefits', 'raw_benefits', 'contact_info', 'actively_hiring', 'applicants', 'created_at', 'job_title', 'requirements', 'full_address', 'city', 'district', 'salary_min', 'salary_max', 'remote_option', 'job_details', 'posted_date', 'scraped_at', 'source_platform', 'source_url', 'is_active', 'is_embedded']
✓ job_category 非空筆數: 7252 / 7252


## 階段五：資料驗證與統計

公司／職缺統計、**job_description** 與 **requirements** 字數分段（150以下｜150~300｜300~500｜500~800｜800~1200｜1200+，向量化建議 150~300）、必填欄位檢查。

In [44]:
# 公司
n_c = len(df_company)
n_c_ind = (df_company["industry"] != "未分類").sum()
print(f"【公司】 總數 {n_c:,} | 有明確產業（非未分類） {n_c_ind:,} | company_size/location 皆 NULL")

# 職缺
n_j = len(df_jobs)
print(f"【職缺】 總數 {n_j:,}")
print(f"  有 job_description: {df_jobs['job_description'].notna().sum():,} | 有 requirements: {df_jobs['requirements'].notna().sum():,}")
print(f"  有 salary_min: {df_jobs['salary_min'].notna().sum():,} | 有 full_address: {df_jobs['full_address'].notna().sum():,}")

# job_description 字數統計（向量化建議 150~300 字）
desc_len = df_jobs["job_description"].fillna("").str.len()
print(f"【job_description 字數】 平均 {desc_len.mean():.0f} 字")
print(f"  150以下: {(desc_len < 150).sum():,} | 150~300: {((desc_len >= 150) & (desc_len <= 300)).sum():,} | 300~500: {((desc_len > 300) & (desc_len <= 500)).sum():,} | 500~800: {((desc_len > 500) & (desc_len <= 800)).sum():,} | 800~1200: {((desc_len > 800) & (desc_len <= 1200)).sum():,} | 1200+: {(desc_len > 1200).sum():,}")

# requirements 字數統計（同向量化建議）
req_len = df_jobs["requirements"].fillna("").str.len()
print(f"【requirements 字數】 平均 {req_len.mean():.0f} 字")
print(f"  150以下: {(req_len < 150).sum():,} | 150~300: {((req_len >= 150) & (req_len <= 300)).sum():,} | 300~500: {((req_len > 300) & (req_len <= 500)).sum():,} | 500~800: {((req_len > 500) & (req_len <= 800)).sum():,} | 800~1200: {((req_len > 800) & (req_len <= 1200)).sum():,} | 1200+: {(req_len > 1200).sum():,}")

# 必填：company_name, job_title, job_description 已於階段四移除空值
missing = df_jobs[["company_name", "job_title", "job_description"]].isna().any(axis=1).sum()
print(f"  必填欄位缺失: {missing} 筆")

【公司】 總數 3,275 | 有明確產業（非未分類） 3,137 | company_size/location 皆 NULL
【職缺】 總數 7,252
  有 job_description: 7,252 | 有 requirements: 7,252
  有 salary_min: 7,252 | 有 full_address: 7,252
【job_description 字數】 平均 521 字
  150以下: 1,679 | 150~300: 1,633 | 300~500: 1,363 | 500~800: 1,205 | 800~1200: 693 | 1200+: 679
【requirements 字數】 平均 217 字
  150以下: 3,433 | 150~300: 2,301 | 300~500: 1,009 | 500~800: 345 | 800~1200: 103 | 1200+: 61
  必填欄位缺失: 0 筆


### 檢閱清理後 JD

以下顯示清理後的 `job_description`、`requirements` 完整內容（前 10 筆），可捲動檢視實際長相。

In [45]:
# 檢閱清理後 JD：完整顯示 job_description，可捲動檢視
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_rows", 20)
display(df_jobs[["company_name", "job_title", "job_description", "requirements"]].head(5))
pd.reset_option("display.max_colwidth")
pd.reset_option("display.max_rows")

,company_name,job_title,job_description,requirements
0,佛教慈濟醫療財團法人台北慈濟醫院,行政-教學部-AI 工程師,【工作內容 / 主要任務】 • 針對五年期「健康台灣深耕計畫」設計與建置AI 驅動之教學與研究整合平台（AI 教研平台），包含資料收集、模型開發、服務部署及維護。 • 開發、優化並部署 大型語言模型（LLM）於地端環境之程式與推論服務，確保資料安全、隱私與效能。 • 其他主管交代之教學、研究相關事項。 【必要技能與條件】 • 已取得或即將取得相關學位（電腦科學、資料科學、電機工程、生醫資訊、統計或相關領域） • 平台開發能力：具 Web/後端/前端整合開發與系統架構規劃經驗，熟悉 API 介面、身分驗證與基本資安知識。 • 大型語言模型地端程式編寫能力：能在本地或私有雲端環境部署、微調及整合 LLM 模型（如 GPT、LLaMA、Falcon 等）。 • AI / ML 能力：熟悉深度學習、NLP、推薦系統或多模態模型；熟悉 Python（TensorFlow / PyTorch）、R 等。 • 熟悉資料視覺化與互動式儀表板（Dash/Streamlit/Power BI） 【薪資待遇】 • 待遇優渥 (依學歷、能力敘薪，月薪六萬至十萬間)，具年終獎金。 • 提供健全學術與跨領域合作環境，國定假日同院方正職人員，且周休二日。 【應徵方式】 請於 2025年10月30日 前，將以下資料寄至 tch41775@tzuchi.com.tw（主旨請註明「應徵AI工程師—姓名」）： 1. 個人履歷（含學經歷、專長、聯絡方式） 2. 代表性著作或程式作品（GitHub連結、論文或專案簡介） 【聯絡人】 黃如薏醫師（Email：tch41775@tzuchi.com.tw；電話：02-66289779-62501）,不拘\n碩士以上\n不拘\n不拘\nAI\nLLM
1,資策會_財團法人資訊工業策進會,【數轉院/智造科技中心】產業策略與發展分析師,1.負責電子製造業相關資訊系統或專案管理工作，涵蓋製程改善、智慧製造、低碳永續等領域。 2.電子製造專案規劃，與計畫進度監控、風險與資源配置，確保專案符合時程與預算。 3.協調廠商、協作夥伴與研發單位的跨部門溝通，解決專案推動過程中的瓶頸與技術挑戰。 4.整理專案成果報告，支援後續審查及客戶交付需求。 5.針對產業痛點提出改善建議，協助政府計畫推動，包含資料盤點、產業共識會議、示範應用與數據分析等工作。,不拘\n大學\n不拘\n不拘\n1.具備電子資訊產業背景，對於產業趨勢與應用場域具備基礎了解與興趣。\n2.良好的邏輯分析能力與報告撰寫能力。\n3.具備跨部門或外部單位溝通與協調能力。
2,海科科技有限公司,【外商】Java Backend Developer (*Mid-level) (Java後端開發工程師) [BIT/Global #Payment 全球支付],【Who We Are?】 Hytech是一個年輕、充滿活力的團隊，專注於推動金融科技行業的企業技術轉型，是全球領先的管理技術諮詢公司。創新思維和扁平化的管理，讓團隊成員以公開、透明的方式自在工作，也為全球客戶提供卓越的商業價值服務。 【Why Join The Team?】 Hytech 團隊在共事的過程中核心技術會與時俱進，即時討論，並且有良好的溝通管道，扁平化管理，任何問題或意見都可以討論及合作解決。密切的與跨國同事團隊交流。我們的工程師不用輪班，更沒有長期加班的惡性文化。 【About the role - Java Backend Engineer】 您準備好在快速變化的支付與金融科技領域迎接高影響力的挑戰了嗎？我們的核心業務圍繞著支付與交易處理，在這個領域中，技術卓越與穩定性至關重要。我們正在尋找一位具備扎實開發技能並熱愛編碼的 Java 工程師加入我們的研發團隊。您將成為開發團隊中不可或缺的一員，參與設計和實現基於 Java 的系統開發。 【The Challenges！】 Hytech Group 正在打造台北團隊致力於成為全球支付技術的核心開發中心，專注於解決來自世界各地客戶的技術挑戰。我們匯聚頂尖人才，運用最前沿的技術，推動支付解決方案，助力全球業務的無縫交易與發展。 - 高效能挑戰： 您將負責處理複雜的開發任務，應對高交易量系統需求，並確保與外部系統的無縫整合。 - 關鍵性系統： 我們的支付服務必須零錯誤且高度穩定，因此我們要求快速回應並能在必要時迅速解決問題。 - 快節奏環境： 我們所處的產業講求速度與精準度——緊湊的時程與緊急專案是我們日常工作的一部分。 - 影響力與責任：您的工作將直接影響產品運營及客戶滿意度。 *此類技術職位隸屬於集團核心研發團隊，為公司長期發展的關鍵推手。所有職位皆為【正職編制】，具備高度穩定性與良好的職涯發展潛力，主要負責關鍵模組開發，【非接案、短期合作或外包／派遣性質】。 --- 【身為團隊的一份子您將負責】 1. Develop and maintain core backend systems to support key business functions. (開發並維護後端核心系統，以支援主要業務功能) 2. Design efficient data structures and scalable code architecture to meet business requirements. (根據業務需求，設計高效數據結構和程式架構) 3. Promote effective cross-team collaboration through clear communication and logical reasoning. (具備清晰且合邏輯的溝通能力，促進跨團隊有效協作) 4. Regularly assess and enhance team skills to meet R&D standards and evolving requirements. (定期評估並提升團隊技能，以滿足研發標準和需求改變) 5. Execute additional tasks as assigned by supervisors to support team goals. (執行主管指派的其他任務，以支援團隊目標),"2年以上\n專科以上\n不拘\n不拘\n軟體程式設計、網路程式設計\nJava、Spring、MySQL\n【期待您具備的能力與特質】\n1. 3+ years of Java development experience, including 1+ years in backend frameworks, Spring Boot or Netty preferred.\n(具有2年以上的Java開發經驗，並且有1年以上的Spring Boot或Netty開發經驗尤佳)\n2. Proficient in Java with a deep understanding of popular frameworks (Spring Boot, Spring Cloud (Eureka, Nacos), Dubbo, Zookeeper, MyBatis, etc.).\n(精通Java，熟悉主流Java框架的基本原理 (如：Spring Boot、Spring Cloud (Eureka、Nacos)、Dubbo、Zookeeper、MyBatis等))\n3. Demonstrates strong independent design and coding abilities; a solid background in distributed and concurrent systems.\n(擁有獨立的設計和編碼能力，並對分佈式系統和並行系統有基礎認識)\n4. Experience with Redis or similar in-memory data structures for high-performance systems.\n(具備Redis或類似內存數據結構工具的使用經驗，以支援高效能系統)\n5. Hands-on experience in designing and implementing RESTful APIs, focusing on performance and scalability.\n(具備設計與實現RESTful API的實務經驗，並專注於性能及可擴展性)\n6. Well-versed in MySQL databases, with experience in MyBatis, Hibernate, or similar database frameworks.\n(具備MySQL數據庫知識，熟悉MyBatis、Hibernate或其他數據庫框架)\n7. Skilled in Linux operations and command-line usage.\n(熟悉Linux操作)\n8. Possesses clear analytical thinking and can handle high-pressure situations effectively.\n(思路清晰，能承受開發壓力)\n9. Highly responsible, detail-oriented, and committed to quality work. \n(具備強烈的責任感，工

## 階段六：準備 Supabase 連線

載入 `.env`（`Erd/.env` 或 `project_url` / `service_role_key`）、建立 Supabase 客戶端、測試連線。

In [46]:
from dotenv import load_dotenv
import os
from pathlib import Path

# 依序嘗試 .env 路徑
for p in [Path("Erd/.env"), Path(".env"), Path("supabase_control/Erd/.env")]:
    if p.exists():
        load_dotenv(p)
        print(f"✓ 載入 {p}")
        break
else:
    load_dotenv()

SUPABASE_URL = os.getenv("SUPABASE_URL") or os.getenv("project_url")
SUPABASE_KEY = os.getenv("SUPABASE_KEY") or os.getenv("SUPABASE_SERVICE_ROLE_KEY") or os.getenv("service_role_key")

if not SUPABASE_URL or not SUPABASE_KEY:
    raise ValueError("請在 .env 設定 project_url + service_role_key 或 SUPABASE_URL + SUPABASE_KEY")

from supabase import create_client
supabase = create_client(SUPABASE_URL, SUPABASE_KEY)

# 測試連線（依實際表名調整，例如 company_info）
_ = supabase.table("company_info").select("company_id").limit(1).execute()
print("✓ Supabase 連線成功")

✓ 載入 Erd\.env
✓ Supabase 連線成功


## 階段七：寫入 Supabase 資料庫

依 ERD 嚴格寫入 `company_info` → `job_posting`。`job_posting` 僅有 **full_address, city, district**（無 `location`），寫入三者。職缺寫入採用 **Upsert 模式**：以 `source_url` 為衝突鍵，已存在則**更新**為較新資料（依 update_date）；無 source_url 的職缺則依 `(company_id, job_title, full_address)` 跳過重複。

In [47]:
import math
import pandas as pd

# 1. 向量化清洗
def clean_dataframe(df):
    # 使用新版 map 取代 applymap
    df = df.map(lambda x: x.strip() if isinstance(x, str) else x)
    df = df.where(pd.notnull(df), None).replace("", None)
    return df

def _json_safe(v):
    """將 nan/NaT 轉 None，確保 JSON 可序列化"""
    if v is None:
        return None
    if isinstance(v, float) and (math.isnan(v) or math.isinf(v)):
        return None
    if hasattr(v, "__float__") and pd.isna(v):
        return None
    return v

df_company = clean_dataframe(df_company)

# 2. 讀取資料庫現有公司，建立 name→id 映射（需設定 limit 避免預設 1000 筆限制）
existing_data = supabase.table("company_info").select("company_name", "company_id").limit(10000).execute()
company_name_to_id = {d["company_name"]: d["company_id"] for d in existing_data.data}

# 3. 準備批量 Payload（包含 job_category）
to_insert = []
for _, row in df_company.iterrows():
    cname = row["company_name"]
    if cname in company_name_to_id:
        continue  # 已存在，跳過

    payload = {
        "company_name": cname,
        "industry": row["industry"],
        "company_size": row.get("company_size"),
        "location": row.get("location"),
        "website": row.get("website"),
        "description": row.get("description"),
    }
    payload = {k: _json_safe(v) for k, v in payload.items()}
    to_insert.append(payload)

# 4. 批量寫入
success_count = 0
fail_count = 0
if to_insert:
    try:
        for i in range(0, len(to_insert), 500):
            batch_data = to_insert[i:i+500]
            ins_res = supabase.table("company_info").insert(batch_data).execute()
            if ins_res.data:
                success_count += len(ins_res.data)
                for d in ins_res.data:
                    company_name_to_id[d["company_name"]] = d["company_id"]
    except Exception as e:
        print(f"❌ 批量寫入發生嚴重錯誤: {e}")
        fail_count = len(to_insert) - success_count

# 5. 重新取得完整 company_name_to_id（供後續 job_posting 使用，需設定 limit 避免預設 1000 筆限制）
_res = supabase.table("company_info").select("company_name", "company_id").limit(10000).execute()
company_name_to_id = {d["company_name"]: d["company_id"] for d in _res.data}

# 6. 最終結果彙報（重新查詢資料庫以取得準確總數，避免 limit 影響）
try:
    _count_res = supabase.table("company_info").select("*", count="exact").limit(1).execute()
    db_total = getattr(_count_res, "count", None) or len(company_name_to_id)
except Exception:
    db_total = len(company_name_to_id)
print(f"--- 處理回報 ---")
print(f"✓ 成功新增: {success_count} 筆")
print(f"⏭️ 已存在跳過: {len(df_company) - len(to_insert)} 筆")
print(f"⚠️ 寫入失敗: {fail_count} 筆")
print(f"📊 資料庫目前總計: {db_total} 家公司")

--- 處理回報 ---
✓ 成功新增: 0 筆
⏭️ 已存在跳過: 3275 筆
⚠️ 寫入失敗: 0 筆
📊 資料庫目前總計: 3275 家公司


In [48]:
import math
import pandas as pd

# 1. 向量化清洗與預處理 (對齊 ERD 資料型態)
def clean_jobs(df):
    # 使用 map 處理字串，並統一空值
    df = df.map(lambda x: x.strip() if isinstance(x, str) else x)
    df = df.where(pd.notnull(df), None)
    
    # 確保薪資為 INT (對齊 ERD: salary_min/max 為 INT)
    df["salary_min"] = pd.to_numeric(df["salary_min"], errors='coerce').fillna(0).astype(int)
    df["salary_max"] = pd.to_numeric(df["salary_max"], errors='coerce').fillna(0).astype(int)
    
    # 處理布林值 (對齊 ERD: is_active 為 BOOLEAN)
    df["is_active"] = df["is_active"].map(lambda x: True if x is None else bool(x))
    return df

df_jobs = clean_jobs(df_jobs)

# 1.5 檢查 company_name_to_id 是否完整（若少於 df_company 家數，會有職缺因找不到公司而被跳過）
n_companies_in_map = len(company_name_to_id)
n_companies_expected = len(df_company)
if n_companies_in_map < n_companies_expected:
    print(f"⚠️ 警告：company_name_to_id 僅 {n_companies_in_map} 家，預期 {n_companies_expected} 家。請先重新執行上方的「寫入 company_info」cell，確保完整載入。")

# 2. 批量獲取現有職缺指紋（僅用於無 source_url 的重複判定；有 source_url 者走 Upsert 會自動更新）
existing_data = supabase.table("job_posting").select("source_url", "job_title", "company_id", "full_address").limit(10000).execute()
db_rows = existing_data.data if existing_data.data else []

existing_fingerprints_addr = {
    (d.get("company_id"), d.get("job_title"), d.get("full_address")) 
    for d in db_rows 
    if not d.get("source_url") and d.get("job_title")
}

# 3. 準備批量 Payload（date/datetime→ISO；nan→None，否則 JSON 序列化失敗）
def _serialize_dt(v):
    if v is None or (hasattr(v, "__float__") and pd.isna(v)):
        return None
    if hasattr(v, "isoformat"):
        return v.isoformat()
    return str(v) if v is not None else None

def _json_safe(v):
    """將 nan/NaT 轉 None、date/datetime 轉 ISO，確保 JSON 可序列化。"""
    if v is None:
        return None
    if isinstance(v, float) and (math.isnan(v) or math.isinf(v)):
        return None
    if hasattr(v, "__float__") and pd.isna(v):
        return None
    if hasattr(v, "isoformat"):
        return v.isoformat()
    if isinstance(v, dict):
        return {k: _json_safe(vv) for k, vv in v.items()}
    return v

to_insert = []
job_skip = 0  # 僅用於無 source_url 的重複跳過
job_skip_no_company = 0  # 因找不到公司 ID 而跳過的職缺（company_name_to_id 未涵蓋時會發生）

for _, row in df_jobs.iterrows():
    cname = row.get("company_name")
    cid = company_name_to_id.get(cname)
    
    # 若找不到公司 ID，跳過（通常為 company_name_to_id 未完整載入，或 df_company 未涵蓋該公司）
    if not cid:
        job_skip_no_company += 1
        continue
        
    src_url = row.get("source_url")
    title = row.get("job_title")
    addr = row.get("full_address")

    # 重複判定：僅對「無 source_url」的職缺跳過（有 source_url 者走 Upsert 會自動更新）
    if not src_url:
        if (cid, title, addr) in existing_fingerprints_addr:
            job_skip += 1
            continue

    # 封裝資料 (嚴格對齊 5.2 JOB_POSTING 欄位)
    payload = {
        "company_id": cid,
        "job_title": title,
        "job_category": row.get("job_category"),  # 新增：職缺類別
        "job_description": row.get("job_description"),
        "requirements": row.get("requirements"),
        "salary_min": int(row["salary_min"]),
        "salary_max": int(row["salary_max"]),
        "full_address": addr,
        "city": row.get("city"), # ERD 5.2
        "district": row.get("district"), # ERD 5.2
        "remote_option": row.get("remote_option"),
        "job_details": row.get("job_details") if isinstance(row.get("job_details"), dict) else None,
        "source_platform": row.get("source_platform") or "104人力銀行",
        "source_url": src_url,
        "posted_date": _serialize_dt(row.get("posted_date")),
        "scraped_at": _serialize_dt(row.get("scraped_at")),
        "is_active": row.get("is_active"),
    }
    payload = {k: _json_safe(v) for k, v in payload.items()}
    to_insert.append(payload)

# 4. 批量 Upsert（以 source_url 為衝突鍵，已存在則更新為較新資料）
job_ok = 0
job_err = 0
if to_insert:
    try:
        for i in range(0, len(to_insert), 500):
            batch_data = to_insert[i:i+500]
            ins_res = supabase.table("job_posting").upsert(
                batch_data,
                on_conflict="source_url",
                ignore_duplicates=False,  # False = 衝突時更新（以 update_date 較新的資料覆蓋）
            ).execute()
            if ins_res.data:
                job_ok += len(ins_res.data)
    except Exception as e:
        print(f"❌ 批量寫入錯誤: {e}")
        job_err = len(to_insert) - job_ok

# 5. 最終結果彙報（重新查詢資料庫以取得準確總數）
try:
    _count_res = supabase.table("job_posting").select("*", count="exact").limit(1).execute()
    job_db_total = getattr(_count_res, "count", None) or (len(db_rows) + job_ok)
except Exception:
    job_db_total = len(db_rows) + job_ok
print(f"--- 職缺處理回報 (Upsert 模式) ---")
print(f"✓ 成功處理（新增/更新）: {job_ok} 筆")
print(f"⏭️ 跳過重複（無 source_url 且已存在）: {job_skip} 筆")
print(f"⏭️ 跳過（找不到公司 ID）: {job_skip_no_company} 筆")
print(f"❌ 寫入失敗: {job_err} 筆")
print(f"📊 資料庫目前職缺總數: {job_db_total:,} 筆")

--- 職缺處理回報 (Upsert 模式) ---
✓ 成功處理（新增/更新）: 7252 筆
⏭️ 跳過重複（無 source_url 且已存在）: 0 筆
⏭️ 跳過（找不到公司 ID）: 0 筆
❌ 寫入失敗: 0 筆
📊 資料庫目前職缺總數: 7,252 筆


## 階段九：提取技能需求 (job_skill_requirement)

從職缺的 `skills`、`tools` 欄位提取技能，建立職缺與技能的多對多關聯，寫入 `job_skill_requirement` 表。

**前置條件**：`skill_master` 表已建立並填入核心技能（約 50-100 個）。

**完整步驟請參照** `cleaner步驟_v2.md` 階段九，包含：
1. 從 skill_master 建立技能映射表（含同義詞）
2. 從 jobs_cleaned.csv 或 jobs_rows.csv 解析 skills、tools 欄位
3. 匹配技能並建立 (job_id, skill_id) 關聯
4. 去重後批次寫入 job_skill_requirement
5. 處理未匹配技能（匯出 unmatched_skills.csv 供手動補充）